# What is this?

This is the code I've used for additional data not shown in the main_project file. Each section will be explained and there will be comments
showing what you will need to change to do the same. The sections are as follows;

- Finding the different data stored for a specific sensor
- Finding the municipality from lon/lat coordinates
- Finding coordinates for a specific municipality

In [1]:
import geopandas as gpd
import reading_data as rd
from shapely.geometry import Point


In [ ]:
#Midlertidig løsning for å oppdatere reading_data
import importlib
importlib.reload(rd)


In [2]:
client_id = '../ignore_me/client_id.txt'
client_secret = '../ignore_me/client_secret.txt'


## Finding the different data stored for a specific sensor

In [3]:
vervarslinga = rd.ReadingData('SN90450', client_id, client_secret)
vervarslinga.get_station_datatypes('1925')


{'air_temperature',
 'best_estimate_mean(air_temperature P1D)',
 'best_estimate_mean(air_temperature P1M)',
 'best_estimate_mean(air_temperature P1Y)',
 'best_estimate_mean(air_temperature P3M)',
 'best_estimate_mean(air_temperature_anomaly P1M 1961_1990)',
 'best_estimate_mean(air_temperature_anomaly P1Y 1961_1990)',
 'best_estimate_mean(air_temperature_anomaly P3M 1961_1990)',
 'best_estimate_mean(air_temperature_anomaly P6M 1961_1990)',
 'boolean_clear_sky_weather(cloud_area_fraction P1D)',
 'boolean_fair_weather(cloud_area_fraction P1D)',
 'boolean_overcast_weather(cloud_area_fraction P1D)',
 'cloud_area_fraction',
 'integral_of_deficit(mean(air_temperature P1D) P1D 17.0)',
 'integral_of_deficit(mean(air_temperature P1D) P1M 17.0)',
 'integral_of_deficit(mean(air_temperature P1D) P1Y 17.0)',
 'integral_of_deficit(mean(air_temperature P1D) P3M 17.0)',
 'integral_of_deficit(mean(air_temperature P1D) P6M 17.0)',
 'integral_of_excess(mean(air_temperature P1D) P1D 0.0)',
 'integral_of_e

## Finding the municipality from lon/lat coordinates

In [4]:
def get_municipality(geojson_path: str, lat: float, lon: float) -> str:
    """
    Given a coordinate (lat, lon), return the municipality name it falls within.

    Args:
        geojson_path (str): Path to GeoJSON file containing municipality boundaries.
        lat (float): Latitude of the coordinate.
        lon (float): Longitude of the coordinate.
    
    Returns:
        str: Name of the municipality containing the coordinate
    """
    # Load GeoJSON file
    gdf = gpd.read_file(geojson_path)

    # Ensure CRS is in a common format (WGS 84)
    if gdf.crs is None:
        gdf.set_crs(epsg=4326, inplace=True)
    elif gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)

    # Create a point from given coordinates
    point = Point(lon, lat)

    # Find the municipality that contains the point
    municipality = gdf[gdf.contains(point)]

    if not municipality.empty:
        return municipality.iloc[0]['name']  # Adjust column name if needed
    return "Please choose coordinates wihin Norway"


In [5]:
get_municipality('../test_files/Kommuner-L.geojson', 69.95, 18.95)


'Tromsø'

In [6]:
get_municipality('../test_files/Kommuner-L.geojson', 0, 0)


'Please choose coordinates wihin Norway'

## Finding coordinates for a specific municipality

In [7]:
def get_municipality_center(geojson_path: str, municipality_name: str) -> tuple:
    """
    Get the central coordinate (latitude, longitude) of a given municipality.

    :param geojson_path: Path to the GeoJSON file containing municipality boundaries.
    :param municipality_name: The name of the municipality.
    :return: (lat, lon) tuple of the central coordinate or None if not found.
    """
    # Load GeoJSON file
    gdf = gpd.read_file(geojson_path)

    # Find the municipality
    municipality = gdf[gdf['name'] == municipality_name]  # Adjust column name if needed

    if not municipality.empty:
        projected_gdf = municipality.to_crs(epsg=25833)
        centroid = projected_gdf.geometry.centroid.iloc[0]# Ensure CRS is in a common format (WGS 84)
        centroid_wgs84 = gpd.GeoSeries([centroid], crs=projected_gdf.crs).to_crs(epsg=4326).iloc[0]
        return (centroid_wgs84.y, centroid_wgs84.x)
    return None


In [8]:
get_municipality_center('../test_files/Kommuner-L.geojson', 'Oslo')


(59.981077776463806, 10.739521598125112)